In [18]:
from PIL import Image
import os
import imageio
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, LeakyReLU
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


In [19]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [20]:
slum_source = '../Mumbai data/preproccesed data/slum/tiles/'
baseline_source = '../Mumbai data/preproccesed data/baseline/tiles/'

In [21]:
slums = []
for name in os.listdir(slum_source):
    im = imageio.imread(slum_source + '/' + name)
    im = im[:,:,:3]
    slums.append(im)

baseline = []
for name in os.listdir(baseline_source):
    im = imageio.imread(baseline_source + '/' + name)
    im = im[:,:,:3]
    baseline.append(im)

In [22]:
slums = np.array(slums)
baseline = np.array(baseline)


In [23]:
baseline = baseline[:slums.shape[0]]

In [24]:
slums.shape, baseline.shape

((1600, 80, 160, 3), (1600, 80, 160, 3))

In [25]:
labels = np.concatenate((np.zeros(len(slums)),np.ones(len(baseline))))

In [26]:
data = np.concatenate((slums, baseline))

In [27]:
X = data
y = labels
y = to_categorical(y)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [29]:
adam = optimizers.Adam(lr = .001)

In [30]:
pd.DataFrame(y)[0].value_counts(normalize=True)

0.0    0.5
1.0    0.5
Name: 0, dtype: float64

In [31]:
mblnet = MobileNetV2(include_top = True, weights = None, input_shape = (X.shape[1:]), classes = 2)

In [32]:
mblnet.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
mblnet.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 2400 samples, validate on 800 samples
Epoch 1/10
2400/2400 [==============================] - 6s 3ms/sample - loss: 0.1425 - accuracy: 0.9396 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0344 - accuracy: 0.9887 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 3/10
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0461 - accuracy: 0.9850 - val_loss: 0.6955 - val_accuracy: 0.5000
Epoch 4/10
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0159 - accuracy: 0.9937 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0181 - accuracy: 0.9937 - val_loss: 0.6941 - val_accuracy: 0.5000
Epoch 6/10
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0048 - accuracy: 0.9979 - val_loss: 0.6988 - val_accuracy: 0.5000
Epoch 7/10
2400/2400 [==============================] - 3s 1ms/sample - loss:

In [34]:
mblnet.predict(X_train)

array([[0.4580652, 0.5419348],
       [0.4580652, 0.5419348],
       [0.4580652, 0.5419348],
       ...,
       [0.4580652, 0.5419348],
       [0.4580652, 0.5419348],
       [0.4580652, 0.5419348]], dtype=float32)